In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
response = requests.get("https://www.c-span.org/search/?sdate=04%2F26%2F2019&edate=06%2F25%2F2019&searchtype=Videos&sort=Most+Recent+Event&text=0&all%5B%5D=joe&all%5B%5D=biden&tagid%5B%5D=246")
doc = BeautifulSoup(response.text)

In [3]:
rows = []
videos = doc.find_all(class_="onevid")
for video in videos:
    row = {}
    row["title"] = video.find(class_="title").text.strip()
    row["date"] = video.find(class_="time").text.strip()
    row["url"] = video.find("a")["href"]
    rows.append(row)
    
print(rows)

[{'title': 'South Carolina Democratic Party Convention: Joe Biden', 'date': 'June 22, 2019', 'url': '//www.c-span.org/video/?461983-20/south-carolina-democratic-party-convention-joe-biden'}, {'title': "Poor People's Campaign Presidential Forum, Joe Biden and Andrew Yang", 'date': 'June 17, 2019', 'url': '//www.c-span.org/video/?461798-2/poor-peoples-campaign-presidential-forum-joe-biden-andrew-yang'}, {'title': 'Former Vice President Joe Biden Campaigns in Mount Pleasant, IowaJoe Biden in Mount Pleasant, Iowa', 'date': 'June 11, 2019', 'url': '//www.c-span.org/video/?461569-1/vice-president-joe-biden-campaigns-mount-pleasant-iowa'}, {'title': 'Joe Biden Holds Campaign Rally in Davenport, IowaJoe Biden Campaign Rally in Davenport, Iowa', 'date': 'June 11, 2019', 'url': '//www.c-span.org/video/?461636-1/joe-biden-holds-campaign-rally-davenport-iowa'}, {'title': 'Joe Biden Launches Presidential Campaign in PhiladelphiaJoe Biden Presidential Campaign Kick-Off', 'date': 'May 18, 2019', 'url

In [4]:
df_biden = pd.DataFrame(rows)

In [5]:
df_biden

,date,title,url
0,"June 22, 2019",South Carolina Democratic Party Convention: Jo...,//www.c-span.org/video/?461983-20/south-caroli...
1,"June 17, 2019","Poor People's Campaign Presidential Forum, Joe...",//www.c-span.org/video/?461798-2/poor-peoples-...
2,"June 11, 2019",Former Vice President Joe Biden Campaigns in M...,//www.c-span.org/video/?461569-1/vice-presiden...
3,"June 11, 2019","Joe Biden Holds Campaign Rally in Davenport, I...",//www.c-span.org/video/?461636-1/joe-biden-hol...
4,"May 18, 2019",Joe Biden Launches Presidential Campaign in Ph...,//www.c-span.org/video/?460548-1/joe-biden-lau...
5,"May 13, 2019",Former Vice President Joe Biden Makes First Ca...,//www.c-span.org/video/?460665-1/vice-presiden...
6,"April 30, 2019","Joe Biden Campaigns in Dubuque, IowaJoe Biden ...",//www.c-span.org/video/?460257-1/joe-biden-cam...
7,"April 29, 2019",Former Vice President Joe Biden Begins White H...,//www.c-span.org/video/?460193-1/vice-presiden...


In [6]:
df_biden = df_biden[df_biden["title"].str.contains("Biden")]

In [7]:
pd.set_option('display.max_colwidth', -1)

In [8]:
df_biden["url"]

0    //www.c-span.org/video/?461983-20/south-carolina-democratic-party-convention-joe-biden         
1    //www.c-span.org/video/?461798-2/poor-peoples-campaign-presidential-forum-joe-biden-andrew-yang
2    //www.c-span.org/video/?461569-1/vice-president-joe-biden-campaigns-mount-pleasant-iowa        
3    //www.c-span.org/video/?461636-1/joe-biden-holds-campaign-rally-davenport-iowa                 
4    //www.c-span.org/video/?460548-1/joe-biden-launches-presidential-campaign-philadelphia         
5    //www.c-span.org/video/?460665-1/vice-president-joe-biden-makes-campaign-stop-hampshire        
6    //www.c-span.org/video/?460257-1/joe-biden-campaigns-dubuque-iowa                              
7    //www.c-span.org/video/?460193-1/vice-president-joe-biden-begins-white-house-bid-pittsburgh    
Name: url, dtype: object

In [9]:
df_biden['slug'] = df_biden.url.str.replace("//www.c-span.org/video","")
df_biden.head()

,date,title,url,slug
0,"June 22, 2019",South Carolina Democratic Party Convention: Joe Biden,//www.c-span.org/video/?461983-20/south-carolina-democratic-party-convention-joe-biden,/?461983-20/south-carolina-democratic-party-convention-joe-biden
1,"June 17, 2019","Poor People's Campaign Presidential Forum, Joe Biden and Andrew Yang",//www.c-span.org/video/?461798-2/poor-peoples-campaign-presidential-forum-joe-biden-andrew-yang,/?461798-2/poor-peoples-campaign-presidential-forum-joe-biden-andrew-yang
2,"June 11, 2019","Former Vice President Joe Biden Campaigns in Mount Pleasant, IowaJoe Biden in Mount Pleasant, Iowa",//www.c-span.org/video/?461569-1/vice-president-joe-biden-campaigns-mount-pleasant-iowa,/?461569-1/vice-president-joe-biden-campaigns-mount-pleasant-iowa
3,"June 11, 2019","Joe Biden Holds Campaign Rally in Davenport, IowaJoe Biden Campaign Rally in Davenport, Iowa",//www.c-span.org/video/?461636-1/joe-biden-holds-campaign-rally-davenport-iowa,/?461636-1/joe-biden-holds-campaign-rally-davenport-iowa
4,"May 18, 2019",Joe Biden Launches Presidential Campaign in PhiladelphiaJoe Biden Presidential Campaign Kick-Off,//www.c-span.org/video/?460548-1/joe-biden-launches-presidential-campaign-philadelphia,/?460548-1/joe-biden-launches-presidential-campaign-philadelphia


In [10]:
row = df_biden.to_dict()

In [11]:
def scrape_page(row):
    url = f"https://www.c-span.org/video{row['slug']}"
    print("Scraping", url)
    
    response = requests.get(url)
    doc = BeautifulSoup(response.text)

    page = {}

    page['location'] = doc.find(itemprop="contentLocation").text.strip()
  
    return pd.Series(page)

In [12]:
scraped_df = df_biden.apply(scrape_page, axis=1)

Scraping https://www.c-span.org/video/?461983-20/south-carolina-democratic-party-convention-joe-biden
Scraping https://www.c-span.org/video/?461983-20/south-carolina-democratic-party-convention-joe-biden
Scraping https://www.c-span.org/video/?461798-2/poor-peoples-campaign-presidential-forum-joe-biden-andrew-yang
Scraping https://www.c-span.org/video/?461569-1/vice-president-joe-biden-campaigns-mount-pleasant-iowa
Scraping https://www.c-span.org/video/?461636-1/joe-biden-holds-campaign-rally-davenport-iowa
Scraping https://www.c-span.org/video/?460548-1/joe-biden-launches-presidential-campaign-philadelphia
Scraping https://www.c-span.org/video/?460665-1/vice-president-joe-biden-makes-campaign-stop-hampshire
Scraping https://www.c-span.org/video/?460257-1/joe-biden-campaigns-dubuque-iowa
Scraping https://www.c-span.org/video/?460193-1/vice-president-joe-biden-begins-white-house-bid-pittsburgh


In [13]:
df_biden = df_biden.merge(scraped_df, left_index= True, right_index= True)

In [14]:
df_biden['date'] =  df_biden['date'].str.replace(",", "")

In [15]:
df_biden['date'] =  pd.to_datetime(df_biden['date'], format='%B %d %Y')

In [16]:
df_biden = df_biden.sort_values(by='date')

In [17]:
df_biden

,date,title,url,slug,location
7,2019-04-29,Former Vice President Joe Biden Begins White House Bid in PittsburghJoe Biden Campaign Rally in Pittsburgh,//www.c-span.org/video/?460193-1/vice-president-joe-biden-begins-white-house-bid-pittsburgh,/?460193-1/vice-president-joe-biden-begins-white-house-bid-pittsburgh,"Pittsburgh, Pennsylvania, United States"
6,2019-04-30,"Joe Biden Campaigns in Dubuque, IowaJoe Biden in Iowa",//www.c-span.org/video/?460257-1/joe-biden-campaigns-dubuque-iowa,/?460257-1/joe-biden-campaigns-dubuque-iowa,"Dubuque, Iowa, United States"
5,2019-05-13,"Former Vice President Joe Biden Makes First Campaign Stop in New HampshireJoe Biden in Hampton, New Hampshire",//www.c-span.org/video/?460665-1/vice-president-joe-biden-makes-campaign-stop-hampshire,/?460665-1/vice-president-joe-biden-makes-campaign-stop-hampshire,"Hampton, New Hampshire, United States"
4,2019-05-18,Joe Biden Launches Presidential Campaign in PhiladelphiaJoe Biden Presidential Campaign Kick-Off,//www.c-span.org/video/?460548-1/joe-biden-launches-presidential-campaign-philadelphia,/?460548-1/joe-biden-launches-presidential-campaign-philadelphia,"Philadelphia, Pennsylvania, United States"
2,2019-06-11,"Former Vice President Joe Biden Campaigns in Mount Pleasant, IowaJoe Biden in Mount Pleasant, Iowa",//www.c-span.org/video/?461569-1/vice-president-joe-biden-campaigns-mount-pleasant-iowa,/?461569-1/vice-president-joe-biden-campaigns-mount-pleasant-iowa,"Mount Pleasant, Iowa, United States"
3,2019-06-11,"Joe Biden Holds Campaign Rally in Davenport, IowaJoe Biden Campaign Rally in Davenport, Iowa",//www.c-span.org/video/?461636-1/joe-biden-holds-campaign-rally-davenport-iowa,/?461636-1/joe-biden-holds-campaign-rally-davenport-iowa,"Davenport, Iowa, United States"
1,2019-06-17,"Poor People's Campaign Presidential Forum, Joe Biden and Andrew Yang",//www.c-span.org/video/?461798-2/poor-peoples-campaign-presidential-forum-joe-biden-andrew-yang,/?461798-2/poor-peoples-campaign-presidential-forum-joe-biden-andrew-yang,"Washington, District of Columbia, United States"
0,2019-06-22,South Carolina Democratic Party Convention: Joe Biden,//www.c-span.org/video/?461983-20/south-carolina-democratic-party-convention-joe-biden,/?461983-20/south-carolina-democratic-party-convention-joe-biden,"Columbia, South Carolina, United States"


In [24]:
df_biden = df_biden.drop(columns='url')

In [26]:
df_biden.to_csv("biden.csv", index=False)